In [52]:
import pandas as pd

In [53]:
def preprocess(filename):
    f = open(filename, "r")
    sentences = f.readlines()  # Get list of sentences in training set

    corpus = ""
    for i in range(len(sentences)):
        sentences[i] = "<s> " + sentences[i].replace("\n", " </s> ") # Add start and end of sentence tag
        corpus = corpus + sentences[i]  # Concatenate each sentence to the corpus

    tokens = corpus.split(" ")  # Get list of individual tokens

    token_count = {}
    for token in tokens: # Count number of occurences of each token
        if token in token_count:
            token_count[token] += 1
        else:
            token_count[token] = 1
    
    for i in range(len(tokens)): # Mark tokens with single occurrence with <unk>
        if token_count[tokens[i]] == 1:
            tokens[i] = "<unk>"

    vocabulary = set(tokens)  # get dictionary of tokens used in corpus
    return vocabulary, tokens

In [54]:
vocabulary, tokens = preprocess("train.txt")
print("Vocabulary size =",len(vocabulary))

Vocabulary size = 23506


In [55]:
def N_gram_probability_distribution(tokens, N):
    frequency = {}
    if N == 1:
        for token in tokens:
            if token == "<s>" or token == "</s>" or token == "<unk>":
                continue
            if token in frequency:
                frequency[token] += 1
            else:
                frequency[token] = 1
    else:
        probability = 1

    x = pd.DataFrame(frequency.items(), columns=["Token", "Count"])
    print(x.head())


N = int(input())
N_gram_probability_distribution(tokens, N)


     Token  Count
0  liberty     23
1      all   1259
2     star     39
3      usa     41
4     sets    426


In [56]:
# sentence = "food"
# sequence = sentence.split(" ")
# N = int(input())
# probability = compute_N_gram_probability(sequence, tokens, N)
# print(probability)

In [57]:
def get_count(sentence, tokens):
    count = 0
    N=len(sentence)
    for i in range(len(tokens)-N+1):
        ok=True
        for j in range(i,i+N):
            if tokens[j]!=sentence[j-i]:
                ok=False
                break
        if ok:
            print(tokens[i:i+N])
            count+=1
    return count

In [60]:
def generate_sentence(sentence, N, vocabulary, tokens):
    sentence = "<s> "+sentence
    sentence = sentence.split(" ")
    sentence = sentence[len(sentence)-N+1:]
    denominator = get_count(sentence, tokens)
    prediction_count = {}
    for word in vocabulary:
        sentence.append(word)
        prediction_count[word]=get_count(sentence,tokens)
        sentence.pop()


In [61]:
generate_sentence("he told me to do", 4,vocabulary,tokens)

['me', 'to', 'do']


KeyboardInterrupt: 